In [1]:
from tensorflow import keras
from keras.optimizers import Adam
import progressbar

In [2]:
import CantStop

environment = CantStop.CantStopEnv(1)
environment.reset()
environment._get_obs()

{'board': (array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
  array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
  array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
  array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])),
 'current_player_index': 0,
 'current_player_progress': array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 'dice_values': array([5, 2, 1, 3])}

In [3]:

optimizer = Adam(learning_rate=0.01)
agent = CantStop.CantStopAgent(optimizer)

In [4]:
batch_size = 128
num_of_episodes = 100
timesteps_per_episode = 1000
agent.q_network.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 1, 10)             590       
                                                                 
 reshape (Reshape)           (None, 10)                0         
                                                                 
 dense (Dense)               (None, 24)                264       
                                                                 
 dense_1 (Dense)             (None, 12)                300       
                                                                 
Total params: 1,154
Trainable params: 1,154
Non-trainable params: 0
_________________________________________________________________


In [5]:
import numpy as np

for e in range(0, num_of_episodes):
    # Reset the enviroment
    state = environment.reset()
    state = agent.encode_state(state)
    
    # Initialize variables
    reward = 0
    terminated = False
    
    if (e % 10 == 0):
        epsilon = 0
    else: 
        epsilon = 0.1
    
    for timestep in range(timesteps_per_episode):
        # Run Action
        
        action = agent.act(state, environment, epsilon)
            
        next_state, reward, terminated = environment.step(agent.decode_action(action)) 
        
        next_state = agent.encode_state(next_state)
        if epsilon != 0:
            agent.store(state, action, reward, next_state, terminated)
        
        state = next_state
        
        if terminated:
            agent.alighn_target_model()
            break
            
    if epsilon !=0 & len(agent.expirience_replay) > batch_size:
        agent.retrain(batch_size)
    

    print("**********************************")
    print("Episode: {} in {} steps".format(e + 1, timestep))
    print("**********************************")

**********************************
Episode: 1 in 34 steps
**********************************
**********************************
Episode: 2 in 30 steps
**********************************
**********************************
Episode: 3 in 24 steps
**********************************
**********************************
Episode: 4 in 29 steps
**********************************
**********************************
Episode: 5 in 38 steps
**********************************
**********************************
Episode: 6 in 32 steps
**********************************
**********************************
Episode: 7 in 28 steps
**********************************
**********************************
Episode: 8 in 30 steps
**********************************
**********************************
Episode: 9 in 31 steps
**********************************
**********************************
Episode: 10 in 33 steps
**********************************
**********************************
Episode: 11 in 26 steps
**********

In [6]:
environment._has_legal_value()

True